In [11]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import random

# --- 1. Environment Setup ---
class CliffWalkingEnv:
    def __init__(self, rows=4, cols=12):
        self.rows, self.cols = rows, cols
        self.start, self.goal = (3, 0), (3, 11)
        self.reset()

    def reset(self):
        self.state = self.start
        return self.state

    def step(self, action):
        r, c = self.state
        if action == 0: r = max(r - 1, 0)
        elif action == 1: r = min(r + 1, self.rows - 1)
        elif action == 2: c = max(c - 1, 0)
        elif action == 3: c = min(c + 1, self.cols - 1)

        next_state = (r, c)
        if r == 3 and 1 <= c <= 10: # Falling off cliff
            return self.start, -100, True
        elif next_state == self.goal:
            return next_state, 0, True
        else:
            self.state = next_state
            return next_state, -1, False

    def get_action_space(self):
        return [0, 1, 2, 3]

In [12]:
# --- 2. Helper Functions ---
def epsilon_greedy(Q, state, actions, epsilon):
    if random.random() < epsilon:
        return random.choice(actions)
    q_vals = Q[state]
    max_q = np.max(q_vals)
    best_actions = [a for a in actions if q_vals[a] == max_q]
    return random.choice(best_actions)

def smooth(x, window=10):
    return np.convolve(x, np.ones(window)/window, mode='valid')

In [13]:
def sarsa(env, episodes=500, alpha=0.5, gamma=1.0, epsilon=0.1):
    Q = defaultdict(lambda: np.zeros(4))
    rewards = []

    for ep in range(episodes):
        state = env.reset()
        # SARSA picks the FIRST action before the loop starts
        action = epsilon_greedy(Q, state, env.get_action_space(), epsilon)
        total_reward = 0
        done = False

        while not done:
            next_state, reward, done = env.step(action)

            # SARSA picks the NEXT action now to use it in the update
            next_action = epsilon_greedy(Q, next_state, env.get_action_space(), epsilon)

            # Update uses the Q-value of the action we ARE going to take
            TD_target = reward + gamma * Q[next_state][next_action]
            Q[state][action] += alpha * (TD_target - Q[state][action])

            state, action = next_state, next_action
            total_reward += reward

        rewards.append(total_reward)
    return Q, rewards

In [14]:
def q_learning(env, episodes=500, alpha=0.5, gamma=1.0, epsilon=0.1):
    Q = defaultdict(lambda: np.zeros(4))
    rewards = []

    for ep in range(episodes):
        state = env.reset()
        total_reward = 0
        done = False

        while not done:
            # Q-Learning picks the current action
            action = epsilon_greedy(Q, state, env.get_action_space(), epsilon)
            next_state, reward, done = env.step(action)

            # Update uses the MAX Q-value (best case), ignoring epsilon-greedy
            best_next_action = np.argmax(Q[next_state])
            TD_target = reward + gamma * Q[next_state][best_next_action]
            Q[state][action] += alpha * (TD_target - Q[state][action])

            state = next_state
            total_reward += reward

        rewards.append(total_reward)
    return Q, rewards

In [17]:
# --- 4. Execution & Comparison ---

env = CliffWalkingEnv()
Q_sarsa, r_sarsa = sarsa(env)
Q_ql, r_ql = q_learning(env)



In [ ]:
# Plotting
plt.figure(figsize=(12, 6))
plt.plot(smooth(r_sarsa), label='SARSA (Safer)', color='blue')
plt.plot(smooth(r_ql), label='Q-Learning (Riskier)', color='orange')
plt.axhline(y=-13, color='r', linestyle='--', label='Optimal Reward (-13)')
plt.xlabel('Episodes')
plt.ylabel('Sum of Rewards during Episode')
plt.title('SARSA vs Q-Learning Performance')
plt.legend()
plt.grid(True)
plt.show()

In [16]:

# --- 5. Path Visualization ---
def visualize_path(Q, env, name):
    print(f"\n--- {name} Path Visualization ---")
    grid = np.full((env.rows, env.cols), '.', dtype=str)
    state = env.start
    grid[env.goal] = 'G'
    grid[env.start] = 'S'
    grid[3, 1:11] = 'C' # Mark the Cliff

    path_steps = 0
    while state != env.goal and path_steps < 50:
        action = np.argmax(Q[state])
        r, c = state
        if action == 0: r -= 1
        elif action == 1: r += 1
        elif action == 2: c -= 1
        elif action == 3: c += 1

        state = (max(0, min(r, env.rows-1)), max(0, min(c, env.cols-1)))
        if grid[state] == '.': grid[state] = '*'
        path_steps += 1

    for row in grid:
        print(' '.join(row))

env_viz = CliffWalkingEnv()
visualize_path(Q_sarsa, env_viz, "SARSA")
visualize_path(Q_ql, env_viz, "Q-Learning")


--- SARSA Path Visualization ---
* * * * * * * * * * . .
* . . . . . . . . * * .
* . . . . . . . . . * *
S C C C C C C C C C C G

--- Q-Learning Path Visualization ---
. . . . . . . . . . . .
. . . . . . . . . . . .
* * * * * * * * * * * *
S C C C C C C C C C C G
